# Murmeln in der Schüssel (Das Prinzip von d'Alembert)

**Anmerkung:** Dies ist eine erweiterte Version des Notebooks für das siebte Video der `Physik mit Python` YouTube-Reihe. Die YouTube Version ist etwas allgemeiner gehalten und sie erhält auch nicht die explizite Behandlung von Zwangskräften.

Wir betrachten die eindimensionale Bewegung von Murmeln in einer Schüssel.

Die Schüssel sei durch einen Parameter in der $x, y$-Ebene darstellbar: $x=x(\theta), z=z(\theta)$. Hierbei ist $\theta = \theta(t)$.

<center><img src="figs/pendel_sketch.png" width=400 height=300></center>

Wir wollen die Bewegungsgleichungen für $\theta(t)$ und damit für $x(\theta)$ und $y(\theta)$ finden.

Mit $\vec{r} = \begin{pmatrix} x \\ z \end{pmatrix}$ lauten die Newtonschen Bewegungsgleichungen für das System:

$$
m \ddot{\vec{r}} = \vec{F}_g + \vec{F}_Z = \begin{pmatrix} 0 \\ -mg \end{pmatrix} + \vec{F}_Z \tag{1}
$$

Wir kennen allerdings $\vec{F}_Z$ nicht! Jedoch verrichten Zwangskräfte im vorliegenden Fall auf den *durch sie erlaubten Bahnen keine physikalische Arbeit* am System:

$$
\vec{r}_{\rm c} = \begin{pmatrix} x_{\rm c} \\ z_{\rm c} \end{pmatrix};\quad W_Z = \int \vec{F}_Z\,\text{d}\vec{r}_{\rm c} = \int \vec{F}_Z\vec{v}_{\rm c}\,\text{d}t \overset{!}{=}0
$$ (Anwendung des [Prinzips von d'Alembert](https://de.wikipedia.org/wiki/D%E2%80%99Alembertsches_Prinzip)). Dies hat $\vec{F}_Z\vec{v}_{\rm c}\equiv 0$ zur Folge.

Aus (1) können wir damit $\vec{F}_Z$ eliminieren:

$$
0 = m \ddot{\vec{r}} - \vec{F}_g - \vec{F}_Z \xRightarrow[]{\text{d'Alembert}} \left( m \ddot{\vec{r}_{\rm c}} - \vec{F}_g - \vec{F}_Z\right)\cdot{\vec{v}_{\rm c}} = \left( m \ddot{\vec{r}_{\rm c}} - \vec{F}_g\right)\cdot\vec{v}_{\rm c} = 0
$$

Aus 

$$
\left( m \ddot{\vec{r}_{\rm c}} - \vec{F}_g\right)\cdot{\vec{v}_{\rm c}} = 0 \tag{2}
$$

leiten wir die Bewegungsgleichung für $\theta(t)$ ab, *ohne* $\vec{F}_Z$ kennen zu müssen!

Wenn man daran interessiert ist, können nach Lösung der Bewegungsgleichungen über (2) die Zwangskräfte mit

$$
\vec{F}_Z = m \ddot{\vec{r_c}} = \vec{F}_g
$$

erhalten werden.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as ma
import sympy as sp
import scipy.integrate as si

sp.init_printing()

## 1. Analytische Bewegungsgleichungen mit SymPy

### Notwendige SymPy Größen

In [ ]:
l, g, t, m = sp.symbols("l, g, t, m", real=True, positive=True)
    
theta = sp.symbols(r"\theta", cls=sp.Function)
theta = theta(t)

In [ ]:
theta_d = sp.diff(theta, t)
theta_dd = sp.diff(theta, t, 2)
theta_dd

### Die Schüsselformen

In [ ]:
form = 'Zykloide'

if form == 'Kreis':
    xc = l * sp.sin(theta)
    zc = -l * sp.cos(theta)

if form == 'Parabel':    
    xc = theta
    zc = l * (2 / sp.pi)**2 * theta**2 - l

if form == 'Zykloide':    
    xc = l / 2 * (2 * theta + sp.sin(2 * theta))
    zc = l / 2 * (-1 - sp.cos(2 * theta))

### Implementation von $\left( m \ddot{\vec{r}_{\rm c}} - \vec{F}_g\right)\cdot{\vec{v}_{\rm c}} = 0$

In [ ]:
# Kurvenvektor
rc = sp.Matrix([xc, zc])
rc

In [ ]:
# Kurvengeschwindigkeit
vc = sp.diff(rc, t)
vc

In [ ]:
# Gewichtskraft
Fg = sp.Matrix([0, -m * g])
Fg

In [ ]:
# Skalarprodukt zwischen Newtongleichung und Geschwindigkeit
ngl = (m * sp.diff(rc, t, 2) - Fg).dot(vc)
ngl = ngl.simplify()
ngl

In [ ]:
# Bewegungsgleichung für theta (zweite Ableitung von theta)
bgl, = sp.solve(ngl, theta_dd)
sp.Eq(theta_dd, bgl)

In [ ]:
## SOLUTION

# Zelle für die Zwangskraft

# Zwangskraft als Funktion von
# theta und theta_d (theta_dd wird eliminiert)
Fz = m * sp.diff(rc, t, 2) - Fg
Fz = sp.simplify(Fz.subs(theta_dd, bgl))
Fz

## 2. Numerische Lösung der Bewegungsgleichung mit SciPy

In [ ]:
# Festlegung der Konstanten
const = {l : 1, g : 9.81, m : 1}
xc = xc.subs(const)
zc = zc.subs(const)
bgl = bgl.subs(const)

In [ ]:
## SOLUTION

# Zelle für die Zwangskraft
Fz = Fz.subs(const)

In [ ]:
# Numpy-Funktionen für Positionen und zweite Ableitung von theta
xc_n = sp.lambdify(theta, xc) 
zc_n = sp.lambdify(theta, zc)
theta_dd_n = sp.lambdify((theta, theta_d), bgl)

In [ ]:
## SOLUTION

# Zelle für die Zwangskraft

# Numpy-Funktion für die Zwangskraft
Fzx_n = sp.lambdify((theta, theta_d), Fz[0])
Fzz_n = sp.lambdify((theta, theta_d), Fz[1])

Wir haben:
$$
\ddot{\theta}(t) = \lambda(\theta, \dot{\theta}(t), t)
$$
Definiere $\omega(t)=\dot{\theta}(t)$ und wir erhalten das gekoppelte DGL-System 1. Ordnung:

\begin{eqnarray}
\dot{\theta}(t) & = & \omega(t) \\
\dot{\omega}(t) & = & \lambda(\theta, \omega(t), t)\\
\end{eqnarray}

Definiere $S=(\theta(t), \omega(t))$ und weiter mit der numerischen Lösung mit `odeint` wie in vorherigen Teilen der Serie.

Schreibe eine Python-Funktion zur Berechnung von

$$
\frac{dS}{dt} = \begin{bmatrix} \dot{\theta}(t) \\ \dot{\omega}(t)\end{bmatrix} = \begin{bmatrix} \omega(t) \\ \lambda(\theta, \omega(t), t)\end{bmatrix}
$$

In [ ]:
def dSdt(S, t):
    theta_n, omega_n = S
    
    return [
        omega_n,   # dtheta / dt
        theta_dd_n(theta_n, omega_n)  # domega / dt
    ]

Bestimme die Zeiten zu denen die DGL gelöst werden soll, lege die Anfangsbedingungen fest und löse die DGL mit `odeint`:

In [ ]:
nframes = 300
nseconds = 10
t_n = np.linspace(0, nseconds, nframes)

# Wir berechnen Lösungen für mehrere 
# Anfangsauslenkungen und speichern diese
# in einer Liste
theta_n_l = []
omega_n_l = []

theta0_vals = [ -sp.pi / 3, -sp.pi / 4, -sp.pi / 6, -sp.pi / 8]

# Kreispendel mit kleinen Auslenkungen
#theta0_vals = [ -sp.pi / 8, -sp.pi / 16, -sp.pi / 32]

for theta0 in theta0_vals:
    S0 = (theta0.n(), 0)
    S = si.odeint(dSdt, t=t_n, y0=S0).T
    theta_n_l.append(S[0])
    omega_n_l.append(S[1])  # Für Zwangskräfte

# Wir wandeln die Python-Listen von theta und omega
# in numpy-Arrays um, damit wir sie als Argumente von
# numpy-Funktionen verwenden können - siehe die direkt
# folgenden Aufrufe:
theta_n_l = np.array(theta_n_l)
omega_n_l = np.array(omega_n_l)

# (x, y)-Positionen der Murmeln für die Animation
xc_n_l = xc_n(theta_n_l)
zc_n_l = zc_n(theta_n_l)

In [ ]:
## SOLUTION

# Zelle für die Zwangskraft

# Zwangskräfte für die Animation mit
# Kräften
Fzx_n_l = Fzx_n(theta_n_l, omega_n_l)
Fzz_n_l = Fzz_n(theta_n_l, omega_n_l)

## 3. Visualisierung der Ergebnisse

### Einfache Plots

In [ ]:
plt.plot(t_n, theta_n_l[0,:], t_n, theta_n_l[1,:], t_n, theta_n_l[2,:])

### Animation der Murmelbewegungen

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9.5, 3.5), constrained_layout=True)

colors = ['blue', 'purple', 'yellow', 'white', 'red', 'lime', 'cyan', 'orange', 'gray']

# Für Plots mit kleinen Winkeln verwenden wir:
#colors = ['red', 'lime', 'cyan', 'orange', 'gray']

for i in range(len(theta_n_l)):
    if i == 0:
        ax.plot(xc_n_l[0,:], zc_n_l[0,:], color='black', lw=1, alpha=0.7, zorder=1)
        ax.fill_between(xc_n_l[0,np.argsort(xc_n_l[0,:])], 
                        zc_n_l[0,np.argsort(xc_n_l[0,:])], 
                        y2=-1, color='black', alpha=0.1, zorder=2)

    # Das folgende Plot-Kommando ist nur für die Legende zuständig!
    ax.plot([], [], 'o', markersize=8, color='black', 
            markerfacecolor=colors[i], label=fr'$\theta_0: {sp.latex(theta0_vals[i])}$')

# Wir realisieren die Murmeln als Scatter-Plot:
scat = ax.scatter(xc_n_l[:,0], zc_n_l[:,0], c=colors[:len(theta_n_l)], 
                  edgecolors='black', s=100, zorder=3)

ax.legend(loc="upper center", ncol=len(xc_n_l), 
          fontsize=15, frameon=True)
ax.set_title(form, fontsize=20)
ax.set_ylim(zc_n_l[0,:].min(), zc_n_l[0,:].max())
ax.set_xlim(xc_n_l[0,:].min(), xc_n_l[0,:].max())
ax.grid()
ax.set_aspect('equal')

# Der Faktor '2' im Nenner von fps streckt das
# Video zeitlich um einen Faktor 2.
fps = nframes / (2 * nseconds)

writer = ma.PillowWriter(fps=fps)

with writer.saving(fig, f'{form}.gif', dpi=100):
    for frame in range(nframes):
       scat.set_offsets(np.column_stack((xc_n_l[:,frame], zc_n_l[:,frame])))
       writer.grab_frame() 

### Animation der Bewegung mit Kräften

In [ ]:
# Welche Lösung (welche Anfangsbedingung) 
# wollen wir animieren
loesung = 0  # maximal: len(theta0_vals) - 1

fig, ax = plt.subplots(1, 1, figsize=(9.5, 3.5), constrained_layout=True)

ax.plot(xc_n_l[0,:], zc_n_l[0,:], color='black', lw=1, alpha=0.7, zorder=1)

ax.set_ylim(zc_n_l[0,:].min() - 0.5, zc_n_l[0,:].max() + 0.2)
ax.set_xlim(xc_n_l[0,:].min() - 0.1, xc_n_l[0,:].max() + 0.2)

ax.xaxis.set_ticklabels([])
ax.yaxis.set_ticklabels([])
ax.xaxis.set_ticks([])
ax.yaxis.set_ticks([])
ax.set_title(f'Kräfte beim {form}-Pendel', fontsize=20)

# Die Murmel
dot, = ax.plot([], [], 'o', markersize=10, color='red')

# Kräfte werden durch Pfeile dargestellt

# Gesamtkraft
arr1 = ax.arrow(0.0, 0.0, 0.0, 0.0, width=0.01, 
                facecolor='black', edgecolor='black',
                label=r'$\vec{F}_{\rm tot}$')

# Zwangskraft
arr2 = ax.arrow(0.0, 0.0, 0.0, 0.0, width=0.01, 
                facecolor='blue', edgecolor='blue',
                label=r'$\vec{F}_{\rm z}$')

# Gewichtskraft
arr3 = ax.arrow(0.0, 0.0, 0.0, 0.0, width=0.01, 
                facecolor='green', edgecolor='green',
                label=r'$\vec{F}_{\rm g}$')

ax.legend(loc='lower center', ncol=3, fontsize=15)
ax.set_aspect('equal')

fps = nframes / (2 * nseconds)

writer = ma.PillowWriter(fps=fps)

# Multilikationsfaktor für die Länge der Kraftpfeile
fac = 0.015

with writer.saving(fig, f'{form}_Kraefte.gif', dpi=100):
    for frame in range(nframes):
        dot.set_data([xc_n_l[loesung, frame]], [zc_n_l[loesung, frame]])

        arr1.set_data(x=xc_n_l[loesung, frame], y=zc_n_l[loesung, frame],
                     dx=2 * Fzx_n_l[loesung][frame] * fac, 
                     dy=2 * (Fzz_n_l[loesung][frame] - const[g]) * fac)
        arr2.set_data(x=xc_n_l[loesung, frame], y=zc_n_l[loesung, frame],
                     dx=2 * Fzx_n_l[loesung][frame] * fac, 
                     dy=2 * Fzz_n_l[loesung][frame] * fac)
        arr3.set_data(x=xc_n_l[loesung, frame], y=zc_n_l[loesung, frame],
                     dx=0.0, 
                     dy=-2 * const[g] * fac)

        writer.grab_frame()